# Demo 2: Simulated Annealing in Travelling Salesman Problem (TSP)

## Step 1: Import Libraries

In [29]:
#Libraries: pandas, math, random
import pandas as pd
import math
import random

## Step 2: Initialize Data

Read the .csv file to get the x and y coordinate of each city

In [30]:
#read cities' coordinate
df = pd.read_csv("./data/problemData.csv")
df.head()

,LocationName,LocationX,LocationY
0,Arad,91,492
1,Bucharest,400,327
2,Craiova,253,288
3,Drobeta,165,299
4,Eforie,562,293


Import the data into a dictionary, format is name:(x, y)

In [31]:
#create dictionary
cities = {}
#import data
cityName = df["LocationName"]   #city name
locationX = df["LocationX"]     #X coordinate
locationY = df["LocationY"]     #Y coordinate
#zip the x and y coordinates into (x,y) format
location = zip(locationX, locationY)
#add to dictionary
for name, coordinate in zip(cityName, location):
    cities.update({name : coordinate})
#show
cities

{'Arad': (91, 492),
 'Bucharest': (400, 327),
 'Craiova': (253, 288),
 'Drobeta': (165, 299),
 'Eforie': (562, 293),
 'Fagaras': (305, 449),
 'Giurgiu': (375, 270),
 'Hirsova': (534, 350),
 'Iasi': (473, 506),
 'Lugoj': (165, 379),
 'Mehadia': (168, 339),
 'Neamt': (406, 537),
 'Oradea': (131, 571),
 'Pitesti': (320, 368),
 'Rimnicu': (233, 410),
 'Sibiu': (207, 457),
 'Timisoara': (94, 410),
 'Urziceni': (456, 350),
 'Vaslui': (509, 444),
 'Zerind': (108, 531)}

## Step 3: Functions

In order to find the best way, we need to calculate the distance first

In [32]:
def distance(city1, city2):
    """
    calculate the Euclidian distance
    :param city1: (x, y)
    :param city2: (x, y)
    :return: Euclidean distance, float
    """
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

With this function, we can calculate the total distance of a path (energy)

In [33]:
def energy(path, cities):
    """
    calculate the total distance of a path
    :param path: city visited order (list)
    :param cities: dictionary of cities
    :return: sum of distance, float
    """
    dist = 0
    for i in range(len(path)):
        dist += distance(cities[path[i]], cities[path[(i + 1) % len(path)]])
    return dist

Generate some different solutions by exchanging two cities (neighbour)

In [34]:
def neighbour(path):
    """
    generate a new path by swapping the positions of two cities
    :param path: current path
    :return: swaped path, list
    """
    neighbours = []
    for _ in range(5):
        tempPath = path[:]
        i, j = random.sample(range(len(path)), 2)               #pick two cities randomly
        tempPath[i], tempPath[j] = tempPath[j], tempPath[i]     #swap picked cities
        neighbours.append(tempPath)
    return neighbours

Define the schedule function

In [35]:
def schedule(k=20, lam=0.005, limit=10000):
    """
    temperature drop function for exponential annealing
    T(t) = k * exp(-lam * t), when t > limit, temperature is 0
    :param k: default temperature
    :param lam: decrease rate
    :param limit: maximum iterations
    :return: a function T(t) that takes an input t and returns the temperature
    """
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)

Core of simulated annealing

In [36]:
def simulatedAnnealing(cities, startCity = "Arad", stopTemp = 0.0001, maxIter = 1000000):
    """
    simulated annealing algorithm for TSP
    :param cities: dictionary of city coordinates
    :param startCity: starting city (the first position on the path)
    :param stopTemp: minimum stopping temperature
    :param maxIter: maximum number of iterations
    :return: best path (list), total distance (float)
    """
    #initial path
    currentPath = list(cities.keys())
    #set the start city
    if startCity and startCity in currentPath:
        #if the start city is set, keep the start city front
        currentPath.remove(startCity)
        random.shuffle(currentPath)
        currentPath = [startCity] + currentPath
    else:
        #else, shuffle all the list
        currentPath = currentPath[:]
        random.shuffle(currentPath)
    #calculate the length of the initial path
    currentEnergy = energy(currentPath, cities)
    #initial the best solution
    bestPath = currentPath[:]
    bestEnergy = currentEnergy
    #initial the temperature function
    T_func = schedule()
    #iteration begin
    for t in range(maxIter):
        T = T_func(t)   #calculate the current temperature
        if T <= stopTemp:   #stop when reach the limit temperature
            break
        #generate a list of neighbours
        neighbours = neighbour(currentPath)
        #pick up a neighbour randomly, then calculate the energy
        newPath = random.choice(neighbours)
        newEnergy = energy(newPath, cities)
        #calculate energy difference
        deltaE = newEnergy - currentEnergy
        #better solutions are accepted directly, worse solutions are accepted according to probability
        if deltaE < 0 or random.random() < math.exp(-deltaE / T):
            currentPath, currentEnergy = newPath, newEnergy #update the current data
            if newEnergy < bestEnergy:  #compare to current best solution
                bestPath, bestEnergy = newPath[:], newEnergy #accept the best solution
    return bestPath, bestEnergy

## Step 4: Run Simulated Annealing

In [37]:
#output method
bestPath, bestEnergy = simulatedAnnealing(cities)
print("Best path: ", bestPath)
print("Best energy: ", bestEnergy)

Best path:  ['Oradea', 'Hirsova', 'Eforie', 'Urziceni', 'Bucharest', 'Pitesti', 'Rimnicu', 'Mehadia', 'Lugoj', 'Sibiu', 'Fagaras', 'Neamt', 'Iasi', 'Vaslui', 'Giurgiu', 'Craiova', 'Drobeta', 'Timisoara', 'Arad', 'Zerind']
Best energy:  2227.412618526964


Notice: Because this algorithm is a random process, each execution will have different output.